In [9]:
Что мы знаем о выборке: 
    0,603167 - доля skilled=0
    0.635872
    0.679764
    0.722986
    0.760106
    
у некоторых tower_damage = 0 т.к. тима проиграла всухую (13к)

SyntaxError: invalid syntax (<ipython-input-9-ccf330fd179a>, line 1)

# Загрузка библиотек и Data

In [3]:
%pylab inline
import pandas
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from catboost import CatBoostClassifier, Pool, CatBoost
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Populating the interactive namespace from numpy and matplotlib


In [4]:
df_train = pandas.read_csv(r'C:\Users\Администратор\Desktop/dota2_skill_train.csv', index_col='id')
df_test = pandas.read_csv(r'C:\Users\Администратор\Desktop/dota2_skill_test.csv', index_col='id')

df_heroes = pandas.read_csv(r'C:\Users\Администратор\Desktop/dota2_heroes.csv', index_col='hero_id')
df_items = pandas.read_csv(r'C:\Users\Администратор\Desktop/dota2_items.csv', index_col='item_id')
df_abilities = pandas.read_csv(r'C:\Users\Администратор\Desktop/dota2_abilities.csv', index_col='ability_id')

In [5]:
X = df_train.drop(columns=['skilled'])
X_test = df_test
#X = df_train.drop(columns=['skilled', 'player_team', 'winner_team','first_blood_time'])
#X_test = df_test.drop(columns=['player_team', 'winner_team','first_blood_time'])
y = df_train['skilled']

In [57]:
df_train.head(), df_train.columns

(    skilled player_team winner_team  duration  pre_game_duration  \
 id                                                                 
 7         1        dire        dire      2140                 90   
 8         0     radiant     radiant      2138                 90   
 12        0     radiant     radiant      3547                 90   
 13        1        dire     radiant      1878                 90   
 14        1        dire     radiant      2232                 90   
 
     first_blood_time  first_blood_claimed  hero_id  hero_pick_order  \
 id                                                                    
 7                129                    0       90                9   
 8                174                    0        5                5   
 12               360                    0       81                7   
 13                28                    0       74                9   
 14               129                    0       14                6   
 
     leav

# Dummies


In [6]:
D = pandas.get_dummies(X, columns = ['hero_id'])
D_test = pandas.get_dummies(X_test, columns = ['hero_id'])
D['player_team'] = D['player_team'].map(lambda s: 1 if s == 'radiant' else 0)
D_test['player_team'] = D_test['player_team'].map(lambda s: 1 if s == 'radiant' else 0)
D['winner_team'] = D['winner_team'].map(lambda s: 1 if s == 'radiant' else 0)
D_test['winner_team'] = D_test['winner_team'].map(lambda s: 1 if s == 'radiant' else 0)

In [17]:
D = D[(D['tower_damage']!=0) & (D['leaver_status']!=1) & (D['hero_healing']==0)]


In [32]:
y_D = df_train[(df_train['tower_damage']!=0) & (df_train['leaver_status']!=1) & (df_train['hero_healing']==0)]['skilled']
y_D

id
13        1
19        0
26        1
29        0
46        0
56        1
57        0
61        1
64        1
69        1
71        1
72        0
73        1
79        0
83        1
88        1
102       0
104       0
108       0
120       0
121       1
122       0
124       0
127       0
129       0
131       1
136       0
139       0
140       0
141       0
         ..
193312    0
193317    0
193326    0
193327    0
193328    0
193339    1
193345    0
193346    1
193348    1
193350    1
193352    1
193353    1
193355    0
193357    0
193359    0
193361    0
193365    0
193366    0
193371    0
193379    0
193380    0
193381    1
193382    1
193383    1
193387    0
193397    1
193398    0
193401    0
193404    0
193409    1
Name: skilled, Length: 56278, dtype: int64

#                                                Нормализация

In [18]:
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
#from sklearn.preprocessing import Normalizer

In [20]:
scaler = RobustScaler()
D_scaled = scaler.fit_transform(D)
D_scaled = pandas.DataFrame(D_scaled, columns=D.columns, index=D.index)
D_scaled_test = scaler.fit_transform(D_test)
D_scaled_test = pandas.DataFrame(D_scaled_test, columns=D_test.columns, index=D_test.index)

#                                            Классификаторы

In [46]:
tree_classifier = XGBClassifier(nb_trees=1500, max_depth=4).fit(D_scaled,y_D)

In [47]:
tree_classifier2 = CatBoostClassifier(n_estimators=900, max_depth =5, loss_function='CrossEntropy', learning_rate = 0.14).fit(D_scaled,y_D)

0:	learn: 0.6716454	total: 2.71s	remaining: 40m 35s
1:	learn: 0.6541156	total: 3.01s	remaining: 22m 32s
2:	learn: 0.6420462	total: 3.21s	remaining: 16m
3:	learn: 0.6316805	total: 3.62s	remaining: 13m 31s
4:	learn: 0.6234498	total: 3.87s	remaining: 11m 32s
5:	learn: 0.6170219	total: 4.01s	remaining: 9m 57s
6:	learn: 0.6116062	total: 4.22s	remaining: 8m 58s
7:	learn: 0.6073006	total: 4.38s	remaining: 8m 8s
8:	learn: 0.6033456	total: 4.59s	remaining: 7m 34s
9:	learn: 0.5998613	total: 4.85s	remaining: 7m 12s
10:	learn: 0.5970726	total: 5.05s	remaining: 6m 48s
11:	learn: 0.5941877	total: 5.18s	remaining: 6m 23s
12:	learn: 0.5917236	total: 5.35s	remaining: 6m 4s
13:	learn: 0.5886900	total: 5.54s	remaining: 5m 50s
14:	learn: 0.5866869	total: 5.86s	remaining: 5m 45s
15:	learn: 0.5845646	total: 6.22s	remaining: 5m 43s
16:	learn: 0.5830149	total: 7.17s	remaining: 6m 12s
17:	learn: 0.5813032	total: 8.12s	remaining: 6m 37s
18:	learn: 0.5800996	total: 8.49s	remaining: 6m 33s
19:	learn: 0.5787027	to

160:	learn: 0.5012526	total: 26.8s	remaining: 2m 2s
161:	learn: 0.5009882	total: 26.8s	remaining: 2m 2s
162:	learn: 0.5005722	total: 26.9s	remaining: 2m 1s
163:	learn: 0.5002975	total: 27s	remaining: 2m 1s
164:	learn: 0.5001295	total: 27s	remaining: 2m
165:	learn: 0.4997994	total: 27.1s	remaining: 1m 59s
166:	learn: 0.4995431	total: 27.2s	remaining: 1m 59s
167:	learn: 0.4992030	total: 27.2s	remaining: 1m 58s
168:	learn: 0.4989826	total: 27.3s	remaining: 1m 58s
169:	learn: 0.4987669	total: 27.4s	remaining: 1m 57s
170:	learn: 0.4983449	total: 27.4s	remaining: 1m 56s
171:	learn: 0.4980361	total: 27.5s	remaining: 1m 56s
172:	learn: 0.4977995	total: 27.6s	remaining: 1m 55s
173:	learn: 0.4975669	total: 27.6s	remaining: 1m 55s
174:	learn: 0.4972867	total: 27.7s	remaining: 1m 54s
175:	learn: 0.4969818	total: 27.9s	remaining: 1m 54s
176:	learn: 0.4967207	total: 27.9s	remaining: 1m 54s
177:	learn: 0.4964798	total: 28s	remaining: 1m 53s
178:	learn: 0.4961923	total: 28.1s	remaining: 1m 53s
179:	le

318:	learn: 0.4664452	total: 38.1s	remaining: 1m 9s
319:	learn: 0.4661862	total: 38.1s	remaining: 1m 9s
320:	learn: 0.4660534	total: 38.2s	remaining: 1m 8s
321:	learn: 0.4658925	total: 38.3s	remaining: 1m 8s
322:	learn: 0.4657477	total: 38.3s	remaining: 1m 8s
323:	learn: 0.4655932	total: 38.4s	remaining: 1m 8s
324:	learn: 0.4654501	total: 38.5s	remaining: 1m 8s
325:	learn: 0.4652923	total: 38.6s	remaining: 1m 7s
326:	learn: 0.4651719	total: 38.6s	remaining: 1m 7s
327:	learn: 0.4649181	total: 38.7s	remaining: 1m 7s
328:	learn: 0.4647468	total: 38.8s	remaining: 1m 7s
329:	learn: 0.4645328	total: 38.8s	remaining: 1m 7s
330:	learn: 0.4643735	total: 38.9s	remaining: 1m 6s
331:	learn: 0.4642076	total: 39s	remaining: 1m 6s
332:	learn: 0.4640756	total: 39.1s	remaining: 1m 6s
333:	learn: 0.4639045	total: 39.2s	remaining: 1m 6s
334:	learn: 0.4636711	total: 39.2s	remaining: 1m 6s
335:	learn: 0.4635121	total: 39.3s	remaining: 1m 5s
336:	learn: 0.4633507	total: 39.4s	remaining: 1m 5s
337:	learn: 0.

478:	learn: 0.4432118	total: 51.1s	remaining: 44.9s
479:	learn: 0.4431089	total: 51.1s	remaining: 44.7s
480:	learn: 0.4429825	total: 51.2s	remaining: 44.6s
481:	learn: 0.4428355	total: 51.3s	remaining: 44.4s
482:	learn: 0.4426928	total: 51.3s	remaining: 44.3s
483:	learn: 0.4425631	total: 51.4s	remaining: 44.2s
484:	learn: 0.4424038	total: 51.4s	remaining: 44s
485:	learn: 0.4423231	total: 51.5s	remaining: 43.9s
486:	learn: 0.4422084	total: 51.6s	remaining: 43.7s
487:	learn: 0.4421056	total: 51.6s	remaining: 43.6s
488:	learn: 0.4418888	total: 51.7s	remaining: 43.5s
489:	learn: 0.4418081	total: 51.8s	remaining: 43.3s
490:	learn: 0.4416761	total: 51.8s	remaining: 43.2s
491:	learn: 0.4415091	total: 51.9s	remaining: 43s
492:	learn: 0.4413875	total: 52s	remaining: 42.9s
493:	learn: 0.4412709	total: 52s	remaining: 42.8s
494:	learn: 0.4411441	total: 52.1s	remaining: 42.6s
495:	learn: 0.4409682	total: 52.2s	remaining: 42.5s
496:	learn: 0.4409041	total: 52.3s	remaining: 42.4s
497:	learn: 0.440782

639:	learn: 0.4245765	total: 1m 1s	remaining: 25s
640:	learn: 0.4245089	total: 1m 1s	remaining: 24.9s
641:	learn: 0.4244070	total: 1m 1s	remaining: 24.8s
642:	learn: 0.4243196	total: 1m 1s	remaining: 24.7s
643:	learn: 0.4241972	total: 1m 1s	remaining: 24.6s
644:	learn: 0.4240910	total: 1m 1s	remaining: 24.5s
645:	learn: 0.4239847	total: 1m 2s	remaining: 24.4s
646:	learn: 0.4239011	total: 1m 2s	remaining: 24.3s
647:	learn: 0.4237650	total: 1m 2s	remaining: 24.2s
648:	learn: 0.4236588	total: 1m 2s	remaining: 24.1s
649:	learn: 0.4235730	total: 1m 2s	remaining: 24s
650:	learn: 0.4234547	total: 1m 2s	remaining: 23.9s
651:	learn: 0.4233659	total: 1m 2s	remaining: 23.8s
652:	learn: 0.4232632	total: 1m 2s	remaining: 23.7s
653:	learn: 0.4231624	total: 1m 2s	remaining: 23.5s
654:	learn: 0.4230565	total: 1m 2s	remaining: 23.4s
655:	learn: 0.4229110	total: 1m 2s	remaining: 23.3s
656:	learn: 0.4227968	total: 1m 2s	remaining: 23.2s
657:	learn: 0.4226452	total: 1m 2s	remaining: 23.1s
658:	learn: 0.42

797:	learn: 0.4086821	total: 1m 12s	remaining: 9.22s
798:	learn: 0.4086171	total: 1m 12s	remaining: 9.13s
799:	learn: 0.4085333	total: 1m 12s	remaining: 9.04s
800:	learn: 0.4084362	total: 1m 12s	remaining: 8.94s
801:	learn: 0.4083479	total: 1m 12s	remaining: 8.85s
802:	learn: 0.4082855	total: 1m 12s	remaining: 8.76s
803:	learn: 0.4081419	total: 1m 12s	remaining: 8.66s
804:	learn: 0.4080812	total: 1m 12s	remaining: 8.57s
805:	learn: 0.4079835	total: 1m 12s	remaining: 8.48s
806:	learn: 0.4078831	total: 1m 12s	remaining: 8.38s
807:	learn: 0.4077675	total: 1m 12s	remaining: 8.29s
808:	learn: 0.4076612	total: 1m 12s	remaining: 8.2s
809:	learn: 0.4075542	total: 1m 12s	remaining: 8.1s
810:	learn: 0.4074467	total: 1m 12s	remaining: 8.01s
811:	learn: 0.4073143	total: 1m 13s	remaining: 7.92s
812:	learn: 0.4071992	total: 1m 13s	remaining: 7.83s
813:	learn: 0.4071140	total: 1m 13s	remaining: 7.73s
814:	learn: 0.4070275	total: 1m 13s	remaining: 7.64s
815:	learn: 0.4069121	total: 1m 13s	remaining: 7

In [48]:
tree_classifier3 = CatBoostClassifier(n_estimators=300, max_depth =8,loss_function='CrossEntropy', learning_rate = 0.2).fit(D_scaled,y_D)

0:	learn: 0.6589889	total: 336ms	remaining: 1m 40s
1:	learn: 0.6370166	total: 610ms	remaining: 1m 30s
2:	learn: 0.6212950	total: 853ms	remaining: 1m 24s
3:	learn: 0.6094982	total: 1.1s	remaining: 1m 21s
4:	learn: 0.5995828	total: 1.31s	remaining: 1m 17s
5:	learn: 0.5918724	total: 1.5s	remaining: 1m 13s
6:	learn: 0.5852818	total: 1.71s	remaining: 1m 11s
7:	learn: 0.5789895	total: 1.88s	remaining: 1m 8s
8:	learn: 0.5749319	total: 2.03s	remaining: 1m 5s
9:	learn: 0.5713615	total: 2.19s	remaining: 1m 3s
10:	learn: 0.5675134	total: 2.34s	remaining: 1m 1s
11:	learn: 0.5641433	total: 2.49s	remaining: 59.8s
12:	learn: 0.5616692	total: 2.63s	remaining: 58.1s
13:	learn: 0.5582539	total: 2.78s	remaining: 56.8s
14:	learn: 0.5542513	total: 2.91s	remaining: 55.3s
15:	learn: 0.5524011	total: 3.04s	remaining: 54s
16:	learn: 0.5508880	total: 3.17s	remaining: 52.9s
17:	learn: 0.5490855	total: 3.3s	remaining: 51.7s
18:	learn: 0.5470253	total: 3.43s	remaining: 50.7s
19:	learn: 0.5453450	total: 3.55s	remai

162:	learn: 0.4176633	total: 22.8s	remaining: 19.2s
163:	learn: 0.4172180	total: 23s	remaining: 19s
164:	learn: 0.4164237	total: 23.1s	remaining: 18.9s
165:	learn: 0.4158193	total: 23.2s	remaining: 18.8s
166:	learn: 0.4149054	total: 23.4s	remaining: 18.6s
167:	learn: 0.4143882	total: 23.5s	remaining: 18.5s
168:	learn: 0.4138200	total: 23.7s	remaining: 18.4s
169:	learn: 0.4131441	total: 23.8s	remaining: 18.2s
170:	learn: 0.4124591	total: 24s	remaining: 18.1s
171:	learn: 0.4120243	total: 24.1s	remaining: 17.9s
172:	learn: 0.4113583	total: 24.2s	remaining: 17.8s
173:	learn: 0.4108556	total: 24.4s	remaining: 17.6s
174:	learn: 0.4102057	total: 24.5s	remaining: 17.5s
175:	learn: 0.4095523	total: 24.6s	remaining: 17.3s
176:	learn: 0.4092590	total: 24.7s	remaining: 17.2s
177:	learn: 0.4085546	total: 24.9s	remaining: 17.1s
178:	learn: 0.4080480	total: 25s	remaining: 16.9s
179:	learn: 0.4076549	total: 25.1s	remaining: 16.8s
180:	learn: 0.4071270	total: 25.3s	remaining: 16.6s
181:	learn: 0.406610

In [49]:
tree_classifier4 = CatBoostClassifier(n_estimators=500, max_depth =6,loss_function='CrossEntropy', learning_rate = 0.23).fit(D_scaled,y_D)

0:	learn: 0.6578023	total: 224ms	remaining: 1m 51s
1:	learn: 0.6352461	total: 472ms	remaining: 1m 57s
2:	learn: 0.6200199	total: 843ms	remaining: 2m 19s
3:	learn: 0.6097117	total: 1.03s	remaining: 2m 7s
4:	learn: 0.6030780	total: 1.22s	remaining: 2m
5:	learn: 0.5963706	total: 1.42s	remaining: 1m 56s
6:	learn: 0.5910635	total: 1.58s	remaining: 1m 51s
7:	learn: 0.5863183	total: 1.75s	remaining: 1m 47s
8:	learn: 0.5826795	total: 1.9s	remaining: 1m 43s
9:	learn: 0.5797963	total: 2.04s	remaining: 1m 40s
10:	learn: 0.5763777	total: 2.19s	remaining: 1m 37s
11:	learn: 0.5736558	total: 2.33s	remaining: 1m 34s
12:	learn: 0.5717786	total: 2.47s	remaining: 1m 32s
13:	learn: 0.5688925	total: 2.6s	remaining: 1m 30s
14:	learn: 0.5652839	total: 2.73s	remaining: 1m 28s
15:	learn: 0.5633988	total: 2.88s	remaining: 1m 27s
16:	learn: 0.5615917	total: 3s	remaining: 1m 25s
17:	learn: 0.5603104	total: 3.11s	remaining: 1m 23s
18:	learn: 0.5583411	total: 3.22s	remaining: 1m 21s
19:	learn: 0.5559405	total: 3.33

163:	learn: 0.4581126	total: 17.2s	remaining: 35.2s
164:	learn: 0.4577423	total: 17.2s	remaining: 35s
165:	learn: 0.4573887	total: 17.3s	remaining: 34.8s
166:	learn: 0.4570202	total: 17.4s	remaining: 34.7s
167:	learn: 0.4566640	total: 17.5s	remaining: 34.5s
168:	learn: 0.4561765	total: 17.5s	remaining: 34.4s
169:	learn: 0.4559188	total: 17.6s	remaining: 34.2s
170:	learn: 0.4555118	total: 17.7s	remaining: 34.1s
171:	learn: 0.4551391	total: 17.8s	remaining: 33.9s
172:	learn: 0.4546431	total: 17.8s	remaining: 33.7s
173:	learn: 0.4543412	total: 17.9s	remaining: 33.6s
174:	learn: 0.4540418	total: 18s	remaining: 33.4s
175:	learn: 0.4536216	total: 18.1s	remaining: 33.3s
176:	learn: 0.4532937	total: 18.1s	remaining: 33.1s
177:	learn: 0.4530613	total: 18.2s	remaining: 33s
178:	learn: 0.4526557	total: 18.3s	remaining: 32.8s
179:	learn: 0.4523161	total: 18.4s	remaining: 32.7s
180:	learn: 0.4518884	total: 18.4s	remaining: 32.5s
181:	learn: 0.4515043	total: 18.5s	remaining: 32.4s
182:	learn: 0.4511

323:	learn: 0.4098291	total: 29.6s	remaining: 16.1s
324:	learn: 0.4095846	total: 29.6s	remaining: 16s
325:	learn: 0.4093790	total: 29.7s	remaining: 15.9s
326:	learn: 0.4090680	total: 29.8s	remaining: 15.8s
327:	learn: 0.4087402	total: 29.9s	remaining: 15.7s
328:	learn: 0.4085526	total: 29.9s	remaining: 15.6s
329:	learn: 0.4084305	total: 30s	remaining: 15.5s
330:	learn: 0.4081631	total: 30.1s	remaining: 15.4s
331:	learn: 0.4078169	total: 30.2s	remaining: 15.3s
332:	learn: 0.4076253	total: 30.2s	remaining: 15.2s
333:	learn: 0.4073679	total: 30.3s	remaining: 15.1s
334:	learn: 0.4071013	total: 30.4s	remaining: 15s
335:	learn: 0.4067814	total: 30.5s	remaining: 14.9s
336:	learn: 0.4066203	total: 30.6s	remaining: 14.8s
337:	learn: 0.4063657	total: 30.6s	remaining: 14.7s
338:	learn: 0.4061460	total: 30.7s	remaining: 14.6s
339:	learn: 0.4058118	total: 30.8s	remaining: 14.5s
340:	learn: 0.4055352	total: 30.9s	remaining: 14.4s
341:	learn: 0.4053195	total: 30.9s	remaining: 14.3s
342:	learn: 0.4050

483:	learn: 0.3741630	total: 42.5s	remaining: 1.4s
484:	learn: 0.3738987	total: 42.5s	remaining: 1.31s
485:	learn: 0.3736400	total: 42.6s	remaining: 1.23s
486:	learn: 0.3733851	total: 42.7s	remaining: 1.14s
487:	learn: 0.3731360	total: 42.8s	remaining: 1.05s
488:	learn: 0.3728479	total: 42.9s	remaining: 964ms
489:	learn: 0.3726205	total: 42.9s	remaining: 876ms
490:	learn: 0.3724267	total: 43s	remaining: 789ms
491:	learn: 0.3722329	total: 43.1s	remaining: 701ms
492:	learn: 0.3720675	total: 43.2s	remaining: 613ms
493:	learn: 0.3718416	total: 43.3s	remaining: 525ms
494:	learn: 0.3715819	total: 43.3s	remaining: 438ms
495:	learn: 0.3712874	total: 43.4s	remaining: 350ms
496:	learn: 0.3709823	total: 43.5s	remaining: 263ms
497:	learn: 0.3707964	total: 43.6s	remaining: 175ms
498:	learn: 0.3705374	total: 43.7s	remaining: 87.5ms
499:	learn: 0.3703043	total: 43.7s	remaining: 0us


# Feature Eng

# Загрузка Data в File

In [141]:
y_pred = tree_classifier2.predict(D_scaled_test)
df_submission = pandas.DataFrame({'skilled': y_pred}, index=df_test.index)
df_submission.to_csv('submission_ensemble.csv')
df_submission

,skilled
id,
0,1.0
1,1.0
4,0.0
5,1.0
9,0.0
10,0.0
11,1.0
17,0.0
24,0.0


In [58]:
#y_pred_1 = tree_classifier.predict_proba(D_scaled)[:,1]
#y_pred_2 = tree_classifier2.predict_proba(D_scaled)[:,1]
#y_pred_3 = tree_classifier3.predict_proba(D_scaled)[:,1]
#y_pred_4 = tree_classifier4.predict_proba(D_scaled)[:,1]
#y_pred_5 = tree_classifier5.predict_proba(D_scaled)[:,1]
#y_pred_6 = tree_classifier6.predict_proba(D_scaled)[:,1]

#y_pred_1 = cross_val_predict(tree_classifier, D_scaled, y_D,cv=2,n_jobs=-1, method = 'predict_proba')
#y_pred_2 = cross_val_predict(tree_classifier2, D_scaled, y_D,cv=2, n_jobs=-1, method = 'predict_proba')
#y_pred_3 = cross_val_predict(tree_classifier3, D_scaled, y_D,cv=2,n_jobs=-1, method = 'predict_proba')
#y_pred_4 = cross_val_predict(tree_classifier4, D_scaled, y_D,cv=2,n_jobs=-1, method = 'predict_proba')


#y_pred = ((y_pred_1/3 + y_pred_2/3+ y_pred_3/3) > 0.5).astype(int)
#y_pred = ((0.5 * y_pred_2 + 0.5 * y_pred_3) > 0.5).astype(int)
y_pred = (((y_pred_2 + y_pred_3 +y_pred_4 +y_pred_1)/4)>0.5).astype(int)
print('Train accuracy:', accuracy_score(y_D, y_pred.T[1]))

Train accuracy: 0.7629802054088631


In [56]:
y_pred[0]

array([0, 1])

In [74]:
dic = {'1': y_pred_1, '2': y_pred_2, '3': y_pred_3, '4': y_pred_4, '5': five_mean_array_test}
Final_test = pandas.DataFrame(data = dic, index=D_scaled_test.index)

In [75]:
Final_test.head()

,1,2,3,4,5
id,,,,,
0,0.370373,0.505432,0.319959,0.430432,0.8
1,0.697342,0.784716,0.766009,0.741930,0.6
4,0.433054,0.489395,0.530367,0.495377,0.6
5,0.436738,0.637075,0.672157,0.553330,0.2
9,0.236239,0.168351,0.126712,0.120786,0.2


In [63]:
decision =  XGBClassifier(nb_trees=1000, max_depth=3).fit(Final ,y_D)

In [65]:
y_pred_final = cross_val_predict(decision, Final, y_D, cv=2,n_jobs=-1)
print('Train accuracy:', accuracy_score(y_D, y_pred_final))

Train accuracy: 0.7653967802693771


In [79]:
y_ff = decision.predict(Final_test)
df_submission = pandas.DataFrame({'skilled': y_ff}, index=df_test.index)
df_submission.to_csv('submission_proba.csv')
df_submission

,skilled
id,
0,0
1,1
4,1
5,1
9,0
10,0
11,1
17,0
24,0


In [78]:
y_ff

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [73]:
y_pred_1

array([0.37037304, 0.69734156, 0.43305394, ..., 0.14358133, 0.38839772,
       0.1897335 ], dtype=float32)

In [187]:
print('Train accuracy:', accuracy_score(y, tree_classifier.predict(D_scaled)))

Train accuracy: 0.8160727338266364


In [67]:
#                                                      Вариант 2+
y_pred_1 = tree_classifier.predict_proba(D_scaled_test)[:, 1]
y_pred_2 = tree_classifier2.predict_proba(D_scaled_test)[:,1]
y_pred_3 = tree_classifier3.predict_proba(D_scaled_test)[:,1]
y_pred_4 = tree_classifier4.predict_proba(D_scaled_test)[:,1]
#y_pred_5 = tree_classifier5.predict_proba(D_scaled_test)[:,1]
#y_pred_6 = tree_classifier6.predict_proba(D_scaled_test)[:,1]

#y_pred = ((y_pred_1/3 + y_pred_2/3+ y_pred_3/3) > 0.5).astype(int)
#y_pred = ((0.5 * y_pred_2 + 0.5 * y_pred_3) > 0.5).astype(int)
#y_pred = (((y_pred_2 + 1.1*y_pred_3 +y_pred_4 +1.1*y_pred_6)/4)>0.5).astype(int)
#df_submission = pandas.DataFrame({'skilled': y_pred}, index=df_test.index)
#df_submission.to_csv('submission_ensemble2.csv')
#df_submission.tail()

#  Валидация

In [10]:
y_pred = cross_val_predict(
    XGBClassifier(nb_trees=2510,colsample_bylevel=0.8,

colsample_bytree=0.6,

eta=1,

gamma=2,

max_depth=10,

min_child_weight=30,

subsample=1),
    X, y,
    cv=5,
    n_jobs=-1,
)

print('Validation accuracy:', accuracy_score(df_train['skilled'], y_pred))

Validation accuracy: 0.6954070751269137


In [43]:
y_pred_2 = cross_val_predict(
    CatBoostClassifier(n_estimators=900, max_depth =5).fit(X,y),
    X, y,
    cv=5,
    n_jobs=-1,
    method='predict_proba'
)[:, 1]

y_pred_3 = cross_val_predict(
    CatBoostClassifier(n_estimators=300, max_depth =6,loss_function='CrossEntropy').fit(X,y),
    X, y,
    cv=5,
    n_jobs=-1,
    method='predict_proba'
)[:, 1]

y_pred = 0.5 * y_pred_2 + 0.5 * y_pred_3

print('Validation accuracy RF:', accuracy_score(df_train['skilled'], y_pred_2 > 0.5))
print('Validation accuracy GB:', accuracy_score(df_train['skilled'], y_pred_3 > 0.5))
print('Validation accuracy RF+GB:', accuracy_score(df_train['skilled'], y_pred > 0.5))

Learning rate set to 0.067723
0:	learn: 0.6834627	total: 128ms	remaining: 1m 55s
1:	learn: 0.6753550	total: 245ms	remaining: 1m 50s
2:	learn: 0.6680418	total: 356ms	remaining: 1m 46s
3:	learn: 0.6614615	total: 463ms	remaining: 1m 43s
4:	learn: 0.6559779	total: 562ms	remaining: 1m 40s
5:	learn: 0.6509705	total: 684ms	remaining: 1m 41s
6:	learn: 0.6463272	total: 785ms	remaining: 1m 40s
7:	learn: 0.6422923	total: 887ms	remaining: 1m 38s
8:	learn: 0.6384611	total: 993ms	remaining: 1m 38s
9:	learn: 0.6350888	total: 1.1s	remaining: 1m 37s
10:	learn: 0.6322453	total: 1.19s	remaining: 1m 36s
11:	learn: 0.6296916	total: 1.3s	remaining: 1m 35s
12:	learn: 0.6270269	total: 1.39s	remaining: 1m 35s
13:	learn: 0.6247156	total: 1.49s	remaining: 1m 34s
14:	learn: 0.6225614	total: 1.59s	remaining: 1m 33s
15:	learn: 0.6205698	total: 1.69s	remaining: 1m 33s
16:	learn: 0.6186932	total: 1.79s	remaining: 1m 33s
17:	learn: 0.6168639	total: 1.9s	remaining: 1m 33s
18:	learn: 0.6151744	total: 2.02s	remaining: 1m

158:	learn: 0.5543101	total: 16.7s	remaining: 1m 17s
159:	learn: 0.5540592	total: 16.8s	remaining: 1m 17s
160:	learn: 0.5538676	total: 16.9s	remaining: 1m 17s
161:	learn: 0.5536923	total: 16.9s	remaining: 1m 17s
162:	learn: 0.5535006	total: 17s	remaining: 1m 17s
163:	learn: 0.5533420	total: 17.1s	remaining: 1m 16s
164:	learn: 0.5531625	total: 17.2s	remaining: 1m 16s
165:	learn: 0.5529160	total: 17.3s	remaining: 1m 16s
166:	learn: 0.5527581	total: 17.4s	remaining: 1m 16s
167:	learn: 0.5525627	total: 17.5s	remaining: 1m 16s
168:	learn: 0.5524424	total: 17.6s	remaining: 1m 16s
169:	learn: 0.5522925	total: 17.7s	remaining: 1m 16s
170:	learn: 0.5521342	total: 17.8s	remaining: 1m 15s
171:	learn: 0.5519681	total: 17.9s	remaining: 1m 15s
172:	learn: 0.5518284	total: 18s	remaining: 1m 15s
173:	learn: 0.5515301	total: 18.1s	remaining: 1m 15s
174:	learn: 0.5513860	total: 18.2s	remaining: 1m 15s
175:	learn: 0.5512269	total: 18.2s	remaining: 1m 15s
176:	learn: 0.5509340	total: 18.4s	remaining: 1m 1

317:	learn: 0.5298565	total: 34.5s	remaining: 1m 3s
318:	learn: 0.5296905	total: 34.6s	remaining: 1m 3s
319:	learn: 0.5295141	total: 34.8s	remaining: 1m 3s
320:	learn: 0.5294242	total: 34.9s	remaining: 1m 2s
321:	learn: 0.5293323	total: 35s	remaining: 1m 2s
322:	learn: 0.5292381	total: 35.1s	remaining: 1m 2s
323:	learn: 0.5291266	total: 35.3s	remaining: 1m 2s
324:	learn: 0.5290111	total: 35.4s	remaining: 1m 2s
325:	learn: 0.5289097	total: 35.6s	remaining: 1m 2s
326:	learn: 0.5288154	total: 35.7s	remaining: 1m 2s
327:	learn: 0.5287161	total: 35.8s	remaining: 1m 2s
328:	learn: 0.5286444	total: 35.9s	remaining: 1m 2s
329:	learn: 0.5285454	total: 36s	remaining: 1m 2s
330:	learn: 0.5284543	total: 36.2s	remaining: 1m 2s
331:	learn: 0.5282624	total: 36.3s	remaining: 1m 2s
332:	learn: 0.5280245	total: 36.4s	remaining: 1m 1s
333:	learn: 0.5279265	total: 36.5s	remaining: 1m 1s
334:	learn: 0.5278645	total: 36.6s	remaining: 1m 1s
335:	learn: 0.5277652	total: 36.8s	remaining: 1m 1s
336:	learn: 0.52

478:	learn: 0.5142624	total: 53.9s	remaining: 47.3s
479:	learn: 0.5141278	total: 54s	remaining: 47.2s
480:	learn: 0.5140241	total: 54.1s	remaining: 47.1s
481:	learn: 0.5139586	total: 54.2s	remaining: 47s
482:	learn: 0.5139162	total: 54.3s	remaining: 46.9s
483:	learn: 0.5138380	total: 54.4s	remaining: 46.7s
484:	learn: 0.5137722	total: 54.5s	remaining: 46.6s
485:	learn: 0.5137200	total: 54.6s	remaining: 46.5s
486:	learn: 0.5136481	total: 54.7s	remaining: 46.4s
487:	learn: 0.5135930	total: 54.8s	remaining: 46.2s
488:	learn: 0.5134907	total: 54.9s	remaining: 46.1s
489:	learn: 0.5133755	total: 55s	remaining: 46s
490:	learn: 0.5132402	total: 55.1s	remaining: 45.9s
491:	learn: 0.5130849	total: 55.1s	remaining: 45.7s
492:	learn: 0.5130231	total: 55.2s	remaining: 45.6s
493:	learn: 0.5129282	total: 55.3s	remaining: 45.5s
494:	learn: 0.5128201	total: 55.4s	remaining: 45.3s
495:	learn: 0.5127591	total: 55.5s	remaining: 45.2s
496:	learn: 0.5127004	total: 55.6s	remaining: 45.1s
497:	learn: 0.512602

637:	learn: 0.5019686	total: 1m 10s	remaining: 29.1s
638:	learn: 0.5018887	total: 1m 10s	remaining: 29s
639:	learn: 0.5017919	total: 1m 11s	remaining: 28.9s
640:	learn: 0.5016375	total: 1m 11s	remaining: 28.8s
641:	learn: 0.5015946	total: 1m 11s	remaining: 28.7s
642:	learn: 0.5015202	total: 1m 11s	remaining: 28.5s
643:	learn: 0.5014607	total: 1m 11s	remaining: 28.4s
644:	learn: 0.5013796	total: 1m 11s	remaining: 28.3s
645:	learn: 0.5013223	total: 1m 11s	remaining: 28.2s
646:	learn: 0.5012752	total: 1m 11s	remaining: 28.1s
647:	learn: 0.5012147	total: 1m 11s	remaining: 28s
648:	learn: 0.5010766	total: 1m 12s	remaining: 27.9s
649:	learn: 0.5010279	total: 1m 12s	remaining: 27.7s
650:	learn: 0.5009702	total: 1m 12s	remaining: 27.6s
651:	learn: 0.5009186	total: 1m 12s	remaining: 27.5s
652:	learn: 0.5008454	total: 1m 12s	remaining: 27.4s
653:	learn: 0.5007811	total: 1m 12s	remaining: 27.3s
654:	learn: 0.5007351	total: 1m 12s	remaining: 27.2s
655:	learn: 0.5006856	total: 1m 12s	remaining: 27.

795:	learn: 0.4920007	total: 1m 26s	remaining: 11.3s
796:	learn: 0.4919603	total: 1m 26s	remaining: 11.2s
797:	learn: 0.4919189	total: 1m 26s	remaining: 11.1s
798:	learn: 0.4918883	total: 1m 26s	remaining: 11s
799:	learn: 0.4918366	total: 1m 27s	remaining: 10.9s
800:	learn: 0.4917871	total: 1m 27s	remaining: 10.8s
801:	learn: 0.4917497	total: 1m 27s	remaining: 10.7s
802:	learn: 0.4916963	total: 1m 27s	remaining: 10.6s
803:	learn: 0.4916529	total: 1m 27s	remaining: 10.4s
804:	learn: 0.4915846	total: 1m 27s	remaining: 10.3s
805:	learn: 0.4915439	total: 1m 27s	remaining: 10.2s
806:	learn: 0.4914998	total: 1m 27s	remaining: 10.1s
807:	learn: 0.4914373	total: 1m 27s	remaining: 10s
808:	learn: 0.4913707	total: 1m 27s	remaining: 9.89s
809:	learn: 0.4913145	total: 1m 28s	remaining: 9.78s
810:	learn: 0.4912772	total: 1m 28s	remaining: 9.67s
811:	learn: 0.4912277	total: 1m 28s	remaining: 9.56s
812:	learn: 0.4911734	total: 1m 28s	remaining: 9.45s
813:	learn: 0.4910777	total: 1m 28s	remaining: 9.3

52:	learn: 0.5489858	total: 7.19s	remaining: 33.5s
53:	learn: 0.5483401	total: 7.3s	remaining: 33.3s
54:	learn: 0.5477949	total: 7.52s	remaining: 33.5s
55:	learn: 0.5474680	total: 7.72s	remaining: 33.7s
56:	learn: 0.5470983	total: 7.84s	remaining: 33.4s
57:	learn: 0.5467134	total: 8.01s	remaining: 33.4s
58:	learn: 0.5460501	total: 8.2s	remaining: 33.5s
59:	learn: 0.5455997	total: 8.33s	remaining: 33.3s
60:	learn: 0.5452606	total: 8.44s	remaining: 33.1s
61:	learn: 0.5447177	total: 8.54s	remaining: 32.8s
62:	learn: 0.5443816	total: 8.67s	remaining: 32.6s
63:	learn: 0.5436375	total: 8.89s	remaining: 32.8s
64:	learn: 0.5431887	total: 9s	remaining: 32.5s
65:	learn: 0.5427403	total: 9.1s	remaining: 32.3s
66:	learn: 0.5422203	total: 9.21s	remaining: 32s
67:	learn: 0.5418037	total: 9.31s	remaining: 31.8s
68:	learn: 0.5412554	total: 9.42s	remaining: 31.5s
69:	learn: 0.5405108	total: 9.54s	remaining: 31.3s
70:	learn: 0.5400639	total: 9.64s	remaining: 31.1s
71:	learn: 0.5391859	total: 9.74s	remai

213:	learn: 0.4910966	total: 27.5s	remaining: 11s
214:	learn: 0.4908088	total: 27.7s	remaining: 10.9s
215:	learn: 0.4906351	total: 27.8s	remaining: 10.8s
216:	learn: 0.4904258	total: 27.9s	remaining: 10.7s
217:	learn: 0.4902357	total: 28.1s	remaining: 10.6s
218:	learn: 0.4896915	total: 28.2s	remaining: 10.4s
219:	learn: 0.4894527	total: 28.4s	remaining: 10.3s
220:	learn: 0.4891803	total: 28.6s	remaining: 10.2s
221:	learn: 0.4890097	total: 28.7s	remaining: 10.1s
222:	learn: 0.4887725	total: 28.8s	remaining: 9.94s
223:	learn: 0.4886466	total: 29s	remaining: 9.82s
224:	learn: 0.4884718	total: 29.1s	remaining: 9.69s
225:	learn: 0.4882584	total: 29.2s	remaining: 9.55s
226:	learn: 0.4881087	total: 29.3s	remaining: 9.42s
227:	learn: 0.4878952	total: 29.4s	remaining: 9.28s
228:	learn: 0.4876771	total: 29.5s	remaining: 9.14s
229:	learn: 0.4874900	total: 29.6s	remaining: 9.01s
230:	learn: 0.4872228	total: 29.7s	remaining: 8.87s
231:	learn: 0.4870342	total: 29.8s	remaining: 8.74s
232:	learn: 0.48

# Knn-алгоритм

In [21]:
from annoy import AnnoyIndex
l = len(D_scaled.columns)
annoy = AnnoyIndex(l)
annoy_data = D_scaled
annoy_index = D_scaled.index.get_values()
five_mean_array = np.array([])

In [22]:
five_mean_array_test = np.array([])

In [23]:
for vec, i in zip(annoy_data.get_values(), annoy_index):
    annoy.add_item(i, vec)

In [24]:
annoy.build(20)

True

In [29]:
vec = D_scaled.get_values()
for i in range(len(D_scaled.get_values())):
    indexes = annoy.get_nns_by_vector(vec[i], 6)
    array = np.array([])
    for j in indexes[1:6]:
        array = np.append(array, df_train[df_train.index == j]['skilled'].get_values())
    five_mean_array = np.append(five_mean_array, array.mean())
    print(indexes[0])

13
19
26
29
46
56
57
61
64
69
71
72
73
79
83
88
102
104
108
120
121
122
124
127
129
131
136
139
140
141
143
146
147
150
157
159
169
171
177
179
182
186
190
196
199
204
205
206
208
213
218
226
233
234
235
237
239
242
246
250
256
258
259
273
276
279
286
293
294
297
308
309
313
317
331
333
334
335
336
337
340
342
349
352
353
354
357
361
363
367
371
379
398
402
415
417
422
426
427
444
447
448
454
456
464
466
468
473
477
479
486
507
187561
514
520
524
528
529
531
539
540
542
546
549
552
573
580
582
587
588
592
594
600
601
604
605
613
615
616
617
622
623
646
651
655
658
668
670
678
680
682
683
685
690
691
692
697
700
712
713
714
717
720
721
727
730
736
739
741
746
749
757
766
773
774
777
781
782
783
786
789
791
793
795
796
804
805
807
808
816
818
822
825
827
828
137619
839
845
846
851
853
859
860
861
864
868
869
870
877
885
891
895
897
898
900
903
907
909
911
913
915
916
921
925
926
927
928
931
938
941
942
943
945
949
950
952
955
960
970
971
972
976
981
984
988
989
990
991
996
1000
1004
1009

5909
5913
5915
5916
5921
5926
5931
5937
5938
5940
5943
5947
5948
5951
5961
5969
5983
5986
5988
5991
5993
5996
6004
32804
6006
6009
6011
6014
6016
6034
6038
65813
6043
6044
6058
6060
6061
6063
6064
6068
6069
6070
6072
6073
6075
6078
6080
6082
6086
6087
6093
6097
6099
6100
6104
6106
6108
6115
6118
6119
6122
6126
6127
6130
6133
6139
6140
6143
6145
6147
6151
6152
6159
6166
6169
6171
6174
6175
6183
6186
6187
6202
6206
6208
6210
6214
6217
6218
6221
6222
6223
6226
6231
6239
6240
6245
6246
6253
6256
6260
6266
6267
6272
6275
6276
6277
6281
6288
6290
6298
6299
6300
6302
6305
6308
6309
6310
6311
6313
6315
6321
6323
6326
6327
6342
6343
6348
6354
6359
6360
6362
6363
6376
6378
6383
6388
6391
6398
6401
6402
6403
6404
6416
6420
6426
6429
6430
6435
6437
6440
6445
6450
6464
6466
6473
6474
6476
6481
6482
6485
6489
6500
6501
6502
6507
6508
6509
6513
6514
6522
6534
6536
6539
6540
6541
6542
6546
6547
6548
6550
6557
6558
6559
6560
6561
6563
6569
6570
6574
6577
6579
6583
6586
6597
6598
6612
6613
6615
6618
661

11414
11420
11422
11424
169206
11436
11443
11455
11464
11467
11470
11477
11481
11482
11485
11488
11489
11490
11498
11501
11504
11512
11513
161879
11516
11530
11532
11537
11539
11540
11545
11551
11557
11558
11562
11567
11568
11569
11571
11573
11581
11584
11585
11586
11590
11591
11592
11593
11598
11602
11605
11609
11612
11613
11618
11628
11629
11632
11633
11635
11638
11642
11643
11644
11660
11664
11674
11675
11683
11687
11689
11694
11696
11698
11705
11707
11710
11713
11714
11717
11718
11719
11720
11721
11724
11728
27218
11732
11736
11747
11754
11762
11764
11772
11777
11781
11783
11785
11788
11789
11791
11795
11796
11800
11805
11816
11817
11821
11826
11838
11839
11842
11854
11855
11859
11865
11868
11871
11872
11873
11879
11881
11884
11885
11890
11891
11894
11897
11909
11911
11912
11921
11922
11923
11927
11928
11932
11937
11945
11946
11950
11951
11954
11955
11960
11964
11967
11969
11970
11975
11978
11983
11984
11985
11987
11990
11991
11994
11995
12001
12002
12003
12005
12007
12010
12013
12

16154
16158
16160
16164
16172
16173
16177
16178
16180
16182
16184
16185
16188
16191
16198
16200
16201
16202
16203
16204
16205
16211
16212
16215
16216
16218
16226
16228
16232
16237
16238
16240
16243
16244
16245
16247
16248
16251
16252
16254
16259
16267
16269
16275
16282
16284
16291
16295
16297
16299
16306
16308
16309
16310
16311
16315
16321
16322
16326
16331
16335
16337
16338
16339
16340
16341
16344
16347
33256
16350
16351
16354
16355
16356
16361
16364
16366
16372
16376
16379
16386
16388
16391
16392
16396
16405
16407
16410
16411
16413
16414
16416
16418
16419
16422
23102
16427
16429
16433
16436
16437
16438
16439
16443
16445
16450
16452
16453
16454
16460
16461
16465
16466
16470
16471
16472
16477
16480
16481
16485
16489
16490
16495
16496
16497
16504
16506
16514
16515
16521
16523
16524
16525
16528
16532
16542
16544
16546
16549
16555
16558
16562
16563
16565
16568
16574
16575
16578
16580
16582
16584
16588
16589
16590
16592
16602
16607
16609
16611
16621
16622
16629
16632
16635
16639
16642
1665

20787
20789
20790
20796
20800
20802
20807
20808
20809
20810
20811
20812
20815
20816
20820
20821
20826
20831
20832
20834
20835
20836
20840
20844
20847
20851
20861
20863
20870
20873
20875
20876
20883
20884
20886
20889
20892
20896
20899
20901
20906
20910
20911
20913
20914
20916
20917
20918
20923
20924
20928
20933
20939
20950
20955
20957
20961
20966
20974
20978
20983
20984
20986
20988
20993
21005
21009
21011
21012
21014
21023
21024
21041
21042
21043
21045
21047
21058
21059
21063
21064
21066
21067
21068
21073
21081
21084
21086
21087
21091
21092
21096
21100
21103
21105
21108
21109
21113
21114
21117
21124
21125
21128
21129
21131
21132
21142
21147
21148
21158
21163
21165
21168
21176
21178
21180
21185
21187
21188
21190
21194
21200
21202
21204
21207
21209
21210
21211
21212
21214
21219
21222
21224
21228
21234
21236
21237
21250
21251
21252
21253
21255
21256
21258
21261
21264
21268
21271
21272
21273
21274
21286
21288
21296
21297
21300
21302
21307
21312
21313
21314
21318
21319
21328
21330
21332
2133

...

173803
173804
173811
173812
173815
173820
173823
173833
173835
173842
173844
173856
173858
173864
173865
173868
173872
173879
173882
173884
173887
173890
173893
173894
173898
173900
173901
173907
173908
173909
173920
173925
173926
173927
173928
173929
173933
173945
173946
173948
173957
173965
173967
173976
173981
173982
173983
173987
173992
173996
174001
174003
174007
174009
174012
174013
174016
174033
174036
174041
174049
174052
174054
174055
174057
174061
174065
174067
174073
174081
174088
174090
174092
174093
174094
174096
174098
174099
174100
174101
174102
174106
174109
174111
174117
174124
174127
174131
174135
174142
174145
174150
174154
174157
174158
174160
174162
174169
174171
174172
174175
174176
174177
174178
174181
174187
174192
174193
174194
174199
174203
174205
174206
174212
174214
174216
174225
174227
174228
174235
174239
174242
174249
174252
174256
174264
174267
174270
174271
174279
174283
174287
174288
174291
174293
174295
174296
174299
174300
174301
174303
174304
174306

177878
177879
177887
177888
177889
177891
177895
177896
177898
177901
177904
177905
177906
177908
177909
177910
177911
177917
177920
177921
177923
177926
177928
177932
177935
177936
177939
177953
177954
177955
177956
177960
177964
177967
177969
177970
177973
177976
177979
177980
177983
177984
177988
177989
177998
178003
178004
178005
178010
178011
178014
178015
178017
178018
178025
178027
178028
178033
178034
178037
178040
178043
178052
178061
178062
178063
178065
178067
178069
178076
178090
178093
178097
178104
178106
178108
178114
178115
178116
178121
178122
178131
178136
178138
178143
178144
178145
178147
178149
178150
178153
178154
178155
178159
178160
178166
178167
178170
178172
178173
178178
178181
178182
178193
178195
178198
178199
178200
178203
178205
178207
178212
178219
178220
178226
178228
178230
178232
178234
178236
178243
178245
178246
178262
178263
178264
178265
178270
178274
178284
178287
178298
178301
178308
178309
178314
178318
178328
178334
178345
178358
178365
178366

181946
181947
181948
181950
181955
181957
181958
181959
181966
181969
181973
181979
181985
181987
181988
181993
181996
181997
181998
182000
182001
182003
182004
182010
182011
182012
182013
57329
182022
182033
182037
182041
182047
182051
182056
182065
182066
182067
182068
182071
19994
182077
182086
182088
182091
182093
182095
182099
182102
182120
182139
182140
182145
182146
182148
182150
182152
182160
182166
182177
182181
182182
182191
182195
182197
182198
182200
182206
182208
182213
182217
182218
182222
182225
182228
182233
182235
182236
182239
182240
182242
182245
182246
182253
182260
182261
182262
182270
182275
182276
182278
182281
182285
182286
182297
182298
182312
182313
182327
182331
182334
182337
182342
182343
182346
182352
182355
182356
182358
182360
182361
182365
182367
182370
182372
182374
182375
182377
182378
182379
182383
182389
182390
182396
182397
182401
182406
182407
182410
182412
182415
182419
182421
182424
182430
182431
182435
182436
182437
182445
182451
182453
182456
1

186053
186056
186057
186062
186064
186068
186075
186083
186084
186093
186094
186096
186100
186103
186104
186109
186111
186119
186120
186123
186129
186134
186136
186139
186142
186144
186149
186150
186153
186159
186167
186168
186169
186171
186181
186184
186186
186187
186188
186189
186194
186196
186197
186199
186204
186205
186207
186208
186209
186212
186218
186219
186220
186221
186222
186225
190373
186242
186250
186265
186270
186271
186272
186274
186277
186279
186288
186293
186297
186305
186307
186309
186318
186329
186332
186337
186342
186343
186346
186351
186355
186357
186362
186370
186372
186373
186374
186377
186382
186384
186387
186392
186393
186394
186396
186399
186405
186406
186407
186411
186413
186422
186429
186430
186431
186432
186433
186436
186437
186438
186441
186443
186445
186452
86834
186460
186463
186465
186470
186473
186478
186482
186488
186492
186494
186499
186500
186501
186504
186505
186510
186511
186524
186526
186527
186530
186533
186535
186536
186537
186538
186544
186549


190171
190184
190185
190186
190189
190192
190198
190203
190207
190211
190216
190220
190224
190226
190227
190229
190230
190233
190235
190237
190238
190241
190248
190254
190264
190267
190270
190272
190276
190278
190281
190283
190286
190287
190290
190291
190298
190300
190303
190304
190306
190309
190315
190322
190323
190324
190326
190335
190341
190350
190351
190355
190357
190359
190363
190365
190369
190370
190373
190377
190387
190390
190391
190395
190396
190400
190403
190411
190417
190419
190424
190425
190431
190432
190434
190437
190438
190453
190455
190461
190462
190468
190475
190479
190481
190483
190488
190492
190493
190497
190504
190506
190508
190510
190511
190514
190515
190521
190523
190524
190525
190531
190536
190542
190543
190544
190548
190550
190551
190557
190558
190562
190565
190568
190570
190571
190575
190578
190579
190580
190583
190584
190585
190587
190597
190614
190617
190620
190622
190627
190631
190636
190643
190644
190650
190652
190656
190661
190663
190664
190669
190672
190677

In [30]:
#D_scaled_test['five_mean'] = five_mean_array_test
D_scaled['five_mean'] = five_mean_array

In [31]:
#five = pandas.DataFrame({'five_mean_test': five_mean_array_test}, index=D_scaled_test.index)
#five.to_csv('five_mean_test.csv')
#five.tail()

five = pandas.DataFrame({'five_mean': five_mean_array}, index=D_scaled.index)
five.to_csv('five_mean.csv')
five.tail()

,five_mean
id,
193397,0.8
193398,0.6
193401,0.0
193404,0.6
193409,0.4


In [34]:
print(accuracy_score(y_D, five.round() ))

0.6404811827001671


In [44]:
D_scaled_test = D_scaled_test.drop(columns = ['five_mean'])